# Build an Agent
## Setup

In [1]:
import yaml
import os

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)

for key in config.keys():
    os.environ[key] = config[key]

## Define tools
Using **Tavily**, a search engine as a tool.

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)

tools = [search]

## Define LLM

In [3]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("gpt-4", model_provider="openai")

In [4]:
model_with_tools = model.bind_tools(tools)

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [5]:
# Now a message that need a tool call
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Amsterdam?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Amsterdam'}, 'id': 'call_Q7dsGdVGAtnmXZqrIYQLsbTl', 'type': 'tool_call'}]


## Create the agent
A *ReAct* agent allows to integrate the model with the tools.

In [6]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc345"}}
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]}, config)
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='670b1f05-16e4-41df-9899-10370bb809d9'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8820eb51-65c2-4c2a-b6a7-e8252575ea57-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [7]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="what's the weather in Amsterdam?")]}, config
)
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='670b1f05-16e4-41df-9899-10370bb809d9'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8820eb51-65c2-4c2a-b6a7-e8252575ea57-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content="what's the weather in Amsterdam?", additional_kwargs={}, response_metadata={}, id='7744cbd0-253d-4cd4-a50a-762b5a516e7e'),


In [8]:
# Checking memory
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="what about tomorrow?")]}, config
)
response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='670b1f05-16e4-41df-9899-10370bb809d9'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8820eb51-65c2-4c2a-b6a7-e8252575ea57-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content="what's the weather in Amsterdam?", additional_kwargs={}, response_metadata={}, id='7744cbd0-253d-4cd4-a50a-762b5a516e7e'),
